# 8. teams

In [2]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

from dotenv import load_dotenv
import os

load_dotenv()

api_version = os.getenv("AZURE_OPENAI_API_VERSION")
api_key = os.getenv("AZURE_OPENAI_KEY")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
azure_openai_chat_completion_client = AzureOpenAIChatCompletionClient(
            model=deployment_name,
            azure_endpoint=azure_endpoint,
            api_version=api_version,
            api_key=api_key,
        )

In [3]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat

# Create an OpenAI model client.
model_client = azure_openai_chat_completion_client

# Define a tool that gets the weather for a city.
async def get_weather(city: str) -> str:
    """Get the weather for a city."""
    return f"The weather in {city} is 72 degrees and Sunny."


# Create an assistant agent.
weather_agent = AssistantAgent(
    "assistant",
    model_client=model_client,
    tools=[get_weather],
    system_message="Respond 'TERMINATE' when task is complete.",
)

# Define a termination condition.
text_termination = TextMentionTermination("TERMINATE")

# Create a single-agent team.
single_agent_team = RoundRobinGroupChat([weather_agent], termination_condition=text_termination)

async def run_team() -> None:
    result = await single_agent_team.run(task="What is the weather in New York?")
    print(result)


# Use `asyncio.run(run_team())` when running in a script.
await run_team()


messages=[TextMessage(id='e729daee-c487-4654-8139-e5fad08a9eb9', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 10, 2, 6, 29, 30, 597728, tzinfo=datetime.timezone.utc), content='What is the weather in New York?', type='TextMessage'), ToolCallRequestEvent(id='bf9925d2-e672-436e-a63c-60e3719c2b09', source='assistant', models_usage=RequestUsage(prompt_tokens=70, completion_tokens=16), metadata={}, created_at=datetime.datetime(2025, 10, 2, 6, 29, 32, 72379, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_nb0B6Abq5IYJIFXpyV8sFn87', arguments='{"city":"New York"}', name='get_weather')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='4fe10353-3de3-4c3b-86fe-7aa4c3378a78', source='assistant', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 10, 2, 6, 29, 32, 73500, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content='The weather in New York is 72 degrees and Sunny.', name='get_weather', call_id='ca

In [4]:
from autogen_agentchat.base import TaskResult


async def run_team_stream() -> None:
    async for message in single_agent_team.run_stream(task="What is the weather in New York?"):
        if isinstance(message, TaskResult):
            print("Stop Reason:", message.stop_reason)
        else:
            print(message)


# Use `asyncio.run(run_team_stream())` when running in a script.
await run_team_stream()



id='fdb29e6d-cb73-4440-93ba-cc9b76471007' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 10, 2, 6, 29, 34, 481517, tzinfo=datetime.timezone.utc) content='What is the weather in New York?' type='TextMessage'
id='59c18e09-a335-4b47-8051-90bb43d7474b' source='assistant' models_usage=RequestUsage(prompt_tokens=136, completion_tokens=16) metadata={} created_at=datetime.datetime(2025, 10, 2, 6, 29, 35, 216449, tzinfo=datetime.timezone.utc) content=[FunctionCall(id='call_7Idy19oN6I9gVEv9Ws7s9SaW', arguments='{"city":"New York"}', name='get_weather')] type='ToolCallRequestEvent'
id='7adaa828-3c95-4f6c-943b-a10cfb1a23e6' source='assistant' models_usage=None metadata={} created_at=datetime.datetime(2025, 10, 2, 6, 29, 35, 217472, tzinfo=datetime.timezone.utc) content=[FunctionExecutionResult(content='The weather in New York is 72 degrees and Sunny.', name='get_weather', call_id='call_7Idy19oN6I9gVEv9Ws7s9SaW', is_error=False)] type='ToolCallExecutionEvent'
id='93a

In [5]:
from autogen_agentchat.ui import Console

# Use `asyncio.run(single_agent_team.reset())` when running in a script.
await single_agent_team.reset()  # Reset the team for the next run.
# Use `asyncio.run(single_agent_team.run_stream(task="What is the weather in Seattle?"))` when running in a script.
await Console(
    single_agent_team.run_stream(task="What is the weather in Seattle?")
)  # Stream the messages to the console.

---------- TextMessage (user) ----------
What is the weather in Seattle?
---------- ToolCallRequestEvent (assistant) ----------
[FunctionCall(id='call_FluYJRZvQDp27MyU7PYYW7sD', arguments='{"city":"Seattle"}', name='get_weather')]
---------- ToolCallExecutionEvent (assistant) ----------
[FunctionExecutionResult(content='The weather in Seattle is 72 degrees and Sunny.', name='get_weather', call_id='call_FluYJRZvQDp27MyU7PYYW7sD', is_error=False)]
---------- ToolCallSummaryMessage (assistant) ----------
The weather in Seattle is 72 degrees and Sunny.
---------- TextMessage (assistant) ----------
The weather in Seattle is 72 degrees and sunny. 

TERMINATE


TaskResult(messages=[TextMessage(id='beaf0d3c-eb69-4cbc-9d6b-873eb99bf5dd', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 10, 2, 6, 29, 38, 833218, tzinfo=datetime.timezone.utc), content='What is the weather in Seattle?', type='TextMessage'), ToolCallRequestEvent(id='ce434989-4c66-4d50-bf3c-fb2c20598e16', source='assistant', models_usage=RequestUsage(prompt_tokens=69, completion_tokens=15), metadata={}, created_at=datetime.datetime(2025, 10, 2, 6, 29, 39, 649522, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_FluYJRZvQDp27MyU7PYYW7sD', arguments='{"city":"Seattle"}', name='get_weather')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='56b5d090-3b61-421d-8f8a-40f70e12a308', source='assistant', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 10, 2, 6, 29, 39, 650486, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content='The weather in Seattle is 72 degrees and Sunny.', name='get_weather', c

In [8]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create an OpenAI model client.
model_client = azure_openai_chat_completion_client
# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    system_message="Provide constructive feedback. Respond with 'APPROVE' to when your feedbacks are addressed.",
)

# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVE")
# Define a termination condition that stops the task after 5 messages.
max_message_termination = MaxMessageTermination(15)
# Combine the termination conditions using the `|`` operator so that the
# task stops when either condition is met.
termination = text_termination | max_message_termination

# Create a team with the primary and critic agents.
reflection_team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=termination)

In [9]:
# Use `asyncio.run(Console(reflection_team.run_stream(task="Write a short poem about fall season.")))` when running in a script.
await Console(
    reflection_team.run_stream(task="가을에 대한 짧은 시를 써보세요.")
)  # Stream the messages to the console.

---------- TextMessage (user) ----------
가을에 대한 짧은 시를 써보세요.
---------- TextMessage (primary) ----------
가을의 바람 살랑이며,  
나뭇잎은 금빛으로 물들고,  
하늘은 높고 푸르러,  
구름은 천천히 흘러간다.  

수확의 기쁨, 풍성한 결실,  
달콤한 사과, 고소한 밤,  
길가에 떨어진 잎사귀처럼,  
우리의 기억도 쌓여만 간다.  

조용히 다가선 가을의 노래,  
차가운 공기 속 따스한 햇살,  
이 순간을 깊이 새기며,  
마음속에 가을을 담아본다.  
---------- TextMessage (critic) ----------
귀한 가을의 모습을 잘 담아낸 시네요! 특히 '나뭇잎은 금빛으로 물들고'와 '수확의 기쁨, 풍성한 결실' 같은 표현이 가을의 풍요로움을 잘 전달하고 있습니다. 

다만, 조금 더 감각적인 이미지나 구체적인 예시를 추가해보면 독자의 감정을 더욱 자극할 수 있을 것 같습니다. 예를 들어, 특정한 나무나 과일의 모습을 묘사해 보거나, 가을의 소리나 냄새를 더해보면 좋을 것 같아요.

전반적으로 잘 쓰셨고, 작은 수정으로 더욱 풍부해질 것 같습니다. 기회가 된다면 한 번 더 다듬어 보시면 좋겠어요!
---------- TextMessage (primary) ----------
고맙습니다! 깊이 있는 피드백 정말 감사드립니다. 가을의 감각을 더욱 풍부하게 담아보도록 노력해보겠습니다. 수정한 내용을 아래에 첨부해볼게요.

---

가을의 바람 살랑이며,  
단풍잎은 나무마다 뜨겁게 물들고,  
느릅나무 아래, 고요한 시간,  
귤향 가득한, 수확의 기쁨.  

밤하늘에 달빛은 부드럽고,  
속삭이는 바람에 실려온  
사과의 아삭함, 단호박의 따스함,  
우리의 마음도 풍성해진다.  

소리 없는 낙엽의 춤사위,  
차가운 공기 속, 담배연기 같은,  
가을의 향기 기억하며,  
마음속에, 이 순간을 깊이 새긴다.

---

이렇게 수정해 보았습니다. 보다

TaskResult(messages=[TextMessage(id='173cf3d2-6af5-4b8e-9f50-ef21fe7c43de', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 10, 2, 6, 29, 51, 231144, tzinfo=datetime.timezone.utc), content='가을에 대한 짧은 시를 써보세요.', type='TextMessage'), TextMessage(id='d767fd05-084f-4a87-94a5-ed0abb0f06f3', source='primary', models_usage=RequestUsage(prompt_tokens=33, completion_tokens=151), metadata={}, created_at=datetime.datetime(2025, 10, 2, 6, 29, 55, 576638, tzinfo=datetime.timezone.utc), content='가을의 바람 살랑이며,  \n나뭇잎은 금빛으로 물들고,  \n하늘은 높고 푸르러,  \n구름은 천천히 흘러간다.  \n\n수확의 기쁨, 풍성한 결실,  \n달콤한 사과, 고소한 밤,  \n길가에 떨어진 잎사귀처럼,  \n우리의 기억도 쌓여만 간다.  \n\n조용히 다가선 가을의 노래,  \n차가운 공기 속 따스한 햇살,  \n이 순간을 깊이 새기며,  \n마음속에 가을을 담아본다.  ', type='TextMessage'), TextMessage(id='658d7a9f-4283-4c0b-86ff-a685e44cebcb', source='critic', models_usage=RequestUsage(prompt_tokens=201, completion_tokens=171), metadata={}, created_at=datetime.datetime(2025, 10, 2, 6, 29, 58, 425632, tzinfo=datetime.timezone

In [10]:
await Console(reflection_team.run_stream(task="중국 당나라 시풍으로 시를 써보세요."))

---------- TextMessage (user) ----------
중국 당나라 시풍으로 시를 써보세요.
---------- TextMessage (primary) ----------
가을빛 드리운 금강산,  
멀리 떠오른 구름이 흘러가고,  
낙엽 아래 금빛 땅이 넓어,  
바람은 사뿐히 노래하네.  

강가에 선 자맥질 잉어,  
파도의 속삭임 가득하니,  
술잔에 담긴 달빛 한 잔,  
추풍의 정취, 마음에 흐르네.  

대숲 속에 삼월의 소리,  
옛기억을 부르는 처량한 하모니,  
한 조각 구름에 찬란한 꿈,  
가을의 길, 끝없이 뻗어나가네.  
---------- TextMessage (critic) ----------
아주 멋진 시를 쓰셨습니다! 중국 당나라 시풍이 잘 반영되어 있고, 특히 자연의 아름다움과 감정을 고요하게 연결짓는 부분이 인상적이네요.

1. **이미지와 비유**: '가을빛 드리운 금강산'이라는 표현이 특히 아름답고 낭만적입니다. 금강산과 가을빛의 조화를 잘 나타내고 있습니다. 다른 구체적인 자연물이나 색채도 추가하여 더욱 풍부한 시각적 이미지를 제공하면 좋겠어요.

2. **리듬과 구조**: 전반적인 리듬과 구조가 안정적입니다. 하지만 한두 구절에서 느낌을 더 살리기 위해 추가적인 비유를 사용할 수 있습니다. 예를 들어, '바람은 사뿐히 노래하네'처럼 풍성한 감정을 더하는 구절이 있으면 좋겠습니다.

3. **감정의 깊이**: 마지막 연에서 '옛기억을 부르는 처량한 하모니'와 같은 감정의 깊이는 매우 돋보입니다. 이전과는 달리 좀 더 애틋한 감정을 추가해 주시면 더욱 감동적인 느낌을 줄 수 있을 것 같습니다.

전체적으로 고풍스러운 미감을 잘 살린 시입니다. 더 발전시키는 방향으로 고려해보시면 좋겠어요. 매우 인상적이었기에 'APPROVE'합니다!


TaskResult(messages=[TextMessage(id='5fc0ece4-4d6f-41cf-ba51-578119378158', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 10, 2, 6, 30, 5, 503000, tzinfo=datetime.timezone.utc), content='중국 당나라 시풍으로 시를 써보세요.', type='TextMessage'), TextMessage(id='e98bc438-847e-4bf2-adb5-6b1d0cb990bf', source='primary', models_usage=RequestUsage(prompt_tokens=821, completion_tokens=157), metadata={}, created_at=datetime.datetime(2025, 10, 2, 6, 30, 8, 236079, tzinfo=datetime.timezone.utc), content='가을빛 드리운 금강산,  \n멀리 떠오른 구름이 흘러가고,  \n낙엽 아래 금빛 땅이 넓어,  \n바람은 사뿐히 노래하네.  \n\n강가에 선 자맥질 잉어,  \n파도의 속삭임 가득하니,  \n술잔에 담긴 달빛 한 잔,  \n추풍의 정취, 마음에 흐르네.  \n\n대숲 속에 삼월의 소리,  \n옛기억을 부르는 처량한 하모니,  \n한 조각 구름에 찬란한 꿈,  \n가을의 길, 끝없이 뻗어나가네.  ', type='TextMessage'), TextMessage(id='2a1a9df2-bbbc-46bb-b9f3-52db358af3f6', source='critic', models_usage=RequestUsage(prompt_tokens=995, completion_tokens=313), metadata={}, created_at=datetime.datetime(2025, 10, 2, 6, 30, 12, 444878, tzinfo=datetime

In [11]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import Handoff
from autogen_agentchat.conditions import HandoffTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat

# Create an OpenAI model client.
model_client = azure_openai_chat_completion_client

# Create a lazy assistant agent that always hands off to the user.
lazy_agent = AssistantAgent(
    "lazy_assistant",
    model_client=model_client,
    handoffs=[Handoff(target="user", message="Transfer to user.")],
    system_message="Always transfer to user when you don't know the answer. Respond 'TERMINATE' when task is complete.",
)

# Define a termination condition that checks for handoff message targetting helper and text "TERMINATE".
handoff_termination = HandoffTermination(target="user")
text_termination = TextMentionTermination("TERMINATE")
termination = handoff_termination | text_termination

# Create a single-agent team.
lazy_agent_team = RoundRobinGroupChat([lazy_agent], termination_condition=termination)

In [12]:
from autogen_agentchat.ui import Console

await Console(lazy_agent_team.run_stream(task="서울 날씨 어때?"),  output_stats=True)

# Handoff to the user before answering the question

await Console(lazy_agent_team.run_stream(task="날씨 맑은데?"))


---------- TextMessage (user) ----------
서울 날씨 어때?
---------- ToolCallRequestEvent (lazy_assistant) ----------
[FunctionCall(id='call_yJ8kb3ZaXXmePc8RAC3186XR', arguments='{}', name='transfer_to_user')]
[Prompt tokens: 66, Completion tokens: 12]
---------- ToolCallExecutionEvent (lazy_assistant) ----------
[FunctionExecutionResult(content='Transfer to user.', name='transfer_to_user', call_id='call_yJ8kb3ZaXXmePc8RAC3186XR', is_error=False)]
---------- HandoffMessage (lazy_assistant) ----------
Transfer to user.
---------- Summary ----------
Number of messages: 4
Finish reason: Handoff to user from lazy_assistant detected.
Total prompt tokens: 66
Total completion tokens: 12
Duration: 0.89 seconds
---------- TextMessage (user) ----------
날씨 맑은데?
---------- TextMessage (lazy_assistant) ----------
맑은 날씨라니 좋네요! 기온이나 바람도 어떤지 궁금하신가요?
---------- ToolCallRequestEvent (lazy_assistant) ----------
[FunctionCall(id='call_v7KMOJdaGPtrXGQW3bRit9sv', arguments='{}', name='transfer_to_user')]
---------

TaskResult(messages=[TextMessage(id='a54ab151-990a-4ac5-9e6b-63822d740825', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 10, 2, 6, 30, 14, 856513, tzinfo=datetime.timezone.utc), content='날씨 맑은데?', type='TextMessage'), TextMessage(id='b23809f3-a935-44a2-b668-67762c1a90bf', source='lazy_assistant', models_usage=RequestUsage(prompt_tokens=102, completion_tokens=27), metadata={}, created_at=datetime.datetime(2025, 10, 2, 6, 30, 15, 953061, tzinfo=datetime.timezone.utc), content='맑은 날씨라니 좋네요! 기온이나 바람도 어떤지 궁금하신가요?', type='TextMessage'), ToolCallRequestEvent(id='f009d7e2-157a-4d5a-ae82-23d5f5a71509', source='lazy_assistant', models_usage=RequestUsage(prompt_tokens=131, completion_tokens=12), metadata={}, created_at=datetime.datetime(2025, 10, 2, 6, 30, 16, 719519, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_v7KMOJdaGPtrXGQW3bRit9sv', arguments='{}', name='transfer_to_user')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='39e2ca